# Лабораторная работа №5

Выполнила Сектименко Ирина, студент группы М8О-410Б-22

## Выбор датасета

Обе задачи я буду решать на одном и том же датасете. Однако выявляться будут разные закономерности.

Данный датасет привлек мое внимание, потому что в современном мире достаточно много людей потребляет кофе и есть много разных точек зрения о его вреде и полезности. По ходу решения задач будут выдвинуты разные гипотезы, которые позволят подтвердить или опровергнуть пользу или вред кофе.

### Классификация

In [1]:
import pandas as pd

In [3]:
class_df = pd.read_csv("synthetic_coffee_health_10000.csv")
class_df = class_df.drop('ID', axis=1)
class_df.head()

,Age,Gender,Country,Coffee_Intake,Caffeine_mg,Sleep_Hours,Sleep_Quality,BMI,Heart_Rate,Stress_Level,Physical_Activity_Hours,Health_Issues,Occupation,Smoking,Alcohol_Consumption
0,40,Male,Germany,3.5,328.1,7.5,Good,24.9,78,Low,14.5,NaN,Other,0,0
1,33,Male,Germany,1.0,94.1,6.2,Good,20.0,67,Low,11.0,NaN,Service,0,0
2,42,Male,Brazil,5.3,503.7,5.9,Fair,22.7,59,Medium,11.2,Mild,Office,0,0
3,53,Male,Germany,2.6,249.2,7.3,Good,24.7,71,Low,6.6,Mild,Other,0,0
4,32,Female,Spain,3.1,298.0,5.3,Fair,24.1,76,Medium,8.5,Mild,Student,0,1


Будем предсказывать уровень стресса человека.

### Регрессия

In [15]:
reg_df = pd.read_csv("synthetic_coffee_health_10000.csv")
reg_df = reg_df.drop('ID', axis=1)
reg_df.head()

,Age,Gender,Country,Coffee_Intake,Caffeine_mg,Sleep_Hours,Sleep_Quality,BMI,Heart_Rate,Stress_Level,Physical_Activity_Hours,Health_Issues,Occupation,Smoking,Alcohol_Consumption
0,40,Male,Germany,3.5,328.1,7.5,Good,24.9,78,Low,14.5,NaN,Other,0,0
1,33,Male,Germany,1.0,94.1,6.2,Good,20.0,67,Low,11.0,NaN,Service,0,0
2,42,Male,Brazil,5.3,503.7,5.9,Fair,22.7,59,Medium,11.2,Mild,Office,0,0
3,53,Male,Germany,2.6,249.2,7.3,Good,24.7,71,Low,6.6,Mild,Other,0,0
4,32,Female,Spain,3.1,298.0,5.3,Fair,24.1,76,Medium,8.5,Mild,Student,0,1


Будем предсказывать сколько часов спит человек, зная информацию о количестве потребляемого кофе, испытываемом стрессе.

### Метрики

Для задачи классификации будем считать accurancy, precision, recall, а для регрессии - MAE и MSE.

## Создание бейзлайна

### Классификация

Поработаем немного с данными. Переведем все категориальные признаки (Gender, Country, Occupation) в численные с помощью one-hot-encoding, потому что все значения этих признаков развноправны.

In [4]:
class_df = pd.get_dummies(class_df, columns=['Gender', 'Country', 'Occupation'], drop_first=True)
class_df.head()

,Age,Coffee_Intake,Caffeine_mg,Sleep_Hours,Sleep_Quality,BMI,Heart_Rate,Stress_Level,Physical_Activity_Hours,Health_Issues,...,Country_South Korea,Country_Spain,Country_Sweden,Country_Switzerland,Country_UK,Country_USA,Occupation_Office,Occupation_Other,Occupation_Service,Occupation_Student
0,40,3.5,328.1,7.5,Good,24.9,78,Low,14.5,NaN,...,False,False,False,False,False,False,False,True,False,False
1,33,1.0,94.1,6.2,Good,20.0,67,Low,11.0,NaN,...,False,False,False,False,False,False,False,False,True,False
2,42,5.3,503.7,5.9,Fair,22.7,59,Medium,11.2,Mild,...,False,False,False,False,False,False,True,False,False,False
3,53,2.6,249.2,7.3,Good,24.7,71,Low,6.6,Mild,...,False,False,False,False,False,False,False,True,False,False
4,32,3.1,298.0,5.3,Fair,24.1,76,Medium,8.5,Mild,...,False,True,False,False,False,False,False,False,False,True


Часть категориальных признаков закодируем label-енкодером. К таким признакам относятся Sleep_Quality, Stress_Level, Health_Issues, потому что чем больше число, тем выше уровень страсса или проблем со здоровьем.

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
label_encoder = LabelEncoder()
class_df['Stress_Level'] = label_encoder.fit_transform(class_df['Stress_Level'])
class_df['Health_Issues'] = label_encoder.fit_transform(class_df['Health_Issues'])
class_df['Sleep_Quality'] = label_encoder.fit_transform(class_df['Sleep_Quality'])
class_df.head()

,Age,Coffee_Intake,Caffeine_mg,Sleep_Hours,Sleep_Quality,BMI,Heart_Rate,Stress_Level,Physical_Activity_Hours,Health_Issues,...,Country_South Korea,Country_Spain,Country_Sweden,Country_Switzerland,Country_UK,Country_USA,Occupation_Office,Occupation_Other,Occupation_Service,Occupation_Student
0,40,3.5,328.1,7.5,2,24.9,78,1,14.5,3,...,False,False,False,False,False,False,False,True,False,False
1,33,1.0,94.1,6.2,2,20.0,67,1,11.0,3,...,False,False,False,False,False,False,False,False,True,False
2,42,5.3,503.7,5.9,1,22.7,59,2,11.2,0,...,False,False,False,False,False,False,True,False,False,False
3,53,2.6,249.2,7.3,2,24.7,71,1,6.6,0,...,False,False,False,False,False,False,False,True,False,False
4,32,3.1,298.0,5.3,1,24.1,76,2,8.5,0,...,False,True,False,False,False,False,False,False,False,True


Поделим датасет на тренировачные и тестирующий.

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
class_train, class_test = train_test_split(class_df, test_size=0.2)

Выделим целевой признак.

In [9]:
class_x_train = class_train.drop('Stress_Level', axis=1)
class_y_train = class_train['Stress_Level']

class_x_test = class_test.drop('Stress_Level', axis=1)
class_y_test = class_test['Stress_Level']

Импортируем модель случайного леса (для задачи классификации) из библиотеки sklearn. Обучим ее на наших данных и спрогнозируем результат на обучающей и тестовой выборке.

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

In [12]:
grad_boost_class = GradientBoostingClassifier()

grad_boost_class.fit(class_x_train, class_y_train)
class_y_train_pred = grad_boost_class.predict(class_x_train)
class_y_test_pred = grad_boost_class.predict(class_x_test)

Оценим качество обученной модели.

In [13]:
from sklearn.metrics import classification_report

In [14]:
print(classification_report(class_y_train, class_y_train_pred))
print(classification_report(class_y_test, class_y_test_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       769
           1       1.00      1.00      1.00      5591
           2       1.00      1.00      1.00      1640

    accuracy                           1.00      8000
   macro avg       1.00      1.00      1.00      8000
weighted avg       1.00      1.00      1.00      8000

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       192
           1       1.00      1.00      1.00      1398
           2       1.00      1.00      1.00       410

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



Как на тренировочном, так и на тестовом, модель прекрасно справилась с предсказанием уровня стресса. Посмотрим, получится ли у нас улучшить метрики.

### Регрессия

По факту у нас та же самая предобработка данных, что и в классификации. Разница только в выделении целевой переменной.

In [16]:
reg_df = pd.get_dummies(reg_df, columns=['Gender', 'Country', 'Occupation'], drop_first=True)
reg_df.head()

,Age,Coffee_Intake,Caffeine_mg,Sleep_Hours,Sleep_Quality,BMI,Heart_Rate,Stress_Level,Physical_Activity_Hours,Health_Issues,...,Country_South Korea,Country_Spain,Country_Sweden,Country_Switzerland,Country_UK,Country_USA,Occupation_Office,Occupation_Other,Occupation_Service,Occupation_Student
0,40,3.5,328.1,7.5,Good,24.9,78,Low,14.5,NaN,...,False,False,False,False,False,False,False,True,False,False
1,33,1.0,94.1,6.2,Good,20.0,67,Low,11.0,NaN,...,False,False,False,False,False,False,False,False,True,False
2,42,5.3,503.7,5.9,Fair,22.7,59,Medium,11.2,Mild,...,False,False,False,False,False,False,True,False,False,False
3,53,2.6,249.2,7.3,Good,24.7,71,Low,6.6,Mild,...,False,False,False,False,False,False,False,True,False,False
4,32,3.1,298.0,5.3,Fair,24.1,76,Medium,8.5,Mild,...,False,True,False,False,False,False,False,False,False,True


In [17]:
label_encoder = LabelEncoder()
reg_df['Stress_Level'] = label_encoder.fit_transform(reg_df['Stress_Level'])
reg_df['Health_Issues'] = label_encoder.fit_transform(reg_df['Health_Issues'])
reg_df['Sleep_Quality'] = label_encoder.fit_transform(reg_df['Sleep_Quality'])
reg_df.head()

,Age,Coffee_Intake,Caffeine_mg,Sleep_Hours,Sleep_Quality,BMI,Heart_Rate,Stress_Level,Physical_Activity_Hours,Health_Issues,...,Country_South Korea,Country_Spain,Country_Sweden,Country_Switzerland,Country_UK,Country_USA,Occupation_Office,Occupation_Other,Occupation_Service,Occupation_Student
0,40,3.5,328.1,7.5,2,24.9,78,1,14.5,3,...,False,False,False,False,False,False,False,True,False,False
1,33,1.0,94.1,6.2,2,20.0,67,1,11.0,3,...,False,False,False,False,False,False,False,False,True,False
2,42,5.3,503.7,5.9,1,22.7,59,2,11.2,0,...,False,False,False,False,False,False,True,False,False,False
3,53,2.6,249.2,7.3,2,24.7,71,1,6.6,0,...,False,False,False,False,False,False,False,True,False,False
4,32,3.1,298.0,5.3,1,24.1,76,2,8.5,0,...,False,True,False,False,False,False,False,False,False,True


In [18]:
reg_train, reg_test = train_test_split(reg_df, test_size=0.2)

In [19]:
reg_x_train = reg_train.drop('Sleep_Hours', axis=1)
reg_y_train = reg_train['Sleep_Hours']

reg_x_test = reg_test.drop('Sleep_Hours', axis=1)
reg_y_test = reg_test['Sleep_Hours']

Предобработка данных завершена. Теперь обучим и протестируем модель.

In [20]:
from sklearn.ensemble import GradientBoostingRegressor

In [22]:
grad_boost_regressor = GradientBoostingRegressor()

grad_boost_regressor.fit(reg_x_train, reg_y_train)
reg_y_train_pred = grad_boost_regressor.predict(reg_x_train)
reg_y_test_pred = grad_boost_regressor.predict(reg_x_test)

In [23]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [24]:
mse_train = mean_squared_error(reg_y_train_pred, reg_y_train)
mse_test = mean_squared_error(reg_y_test_pred, reg_y_test)
print(f"Среднеквадратичная ошибка трейна: {mse_train}")
print(f"Среднеквадратичная ошибка теста: {mse_test}")

mae_train = mean_absolute_error(reg_y_train_pred, reg_y_train)
mae_test = mean_absolute_error(reg_y_test_pred, reg_y_test)
print(f"Средняя абсолютная ошибка трейна: {mae_train}")
print(f"Средняя абсолютная ошибка теста: {mae_test}")

Среднеквадратичная ошибка трейна: 0.22703089394330425
Среднеквадратичная ошибка теста: 0.2445149871851548
Средняя абсолютная ошибка трейна: 0.3886703173380993
Средняя абсолютная ошибка теста: 0.4041559225199585


Ошибка на трейне и тесте хорошо не сильно отличаются друг от друга. Да и в целом ошибка достаточно маленькая.

## Улучшение бейзлайна

### Классификация

Без всякий улучшений градиентный бустинг прекрасно справился со своей задачей на тесте и трейне.

### Регрессия

В градиентном бустинге деревья обучаются последовательно, каждое следующее пытается исправить ошибки предыдущих. На каждой итерации обучается модель на остатках (антиградиентах) функции потерь текущей композиции. Вес каждого дерева фиксируется шагом обучения. Поэтому в конечном итоге мы получаем более точную и совершенную модель.

Градиентный бустинг сам по себе прекрасно работает, в моем случае очистка датасета не приведет к улучшению метрик, так как данная модель достаточно устойчива к данным (линейным зависимостям, выбросам, нормализации так же бессмысленны).

## Имплементация алгоритма

### Классификация

In [29]:
import numpy as np
from collections import Counter
from typing import List

In [30]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class MyDecisionTreeRegressor:
    def __init__(self, max_depth=5, min_samples_split=2, min_samples_leaf=1, max_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.root = None

    def fit(self, X, y):
        X, y = np.array(X, dtype=np.float64), np.array(y, dtype=np.float64)
        self.root = self._grow_tree(X, y, depth=0)

    def _grow_tree(self, X, y, depth):
        n_samples, n_features = X.shape
        if (depth >= self.max_depth or
            n_samples < self.min_samples_split or
            len(np.unique(y)) == 1):
            leaf_value = np.mean(y)
            return Node(value=leaf_value)

        best_feature, best_threshold = self._best_split(X, y, n_features)
        if best_feature is None:
            leaf_value = np.mean(y)
            return Node(value=leaf_value)

        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
        if len(left_idxs) < self.min_samples_leaf or len(right_idxs) < self.min_samples_leaf:
            leaf_value = np.mean(y)
            return Node(value=leaf_value)

        left_child = self._grow_tree(X[left_idxs], y[left_idxs], depth + 1)
        right_child = self._grow_tree(X[right_idxs], y[right_idxs], depth + 1)

        return Node(feature=best_feature, threshold=best_threshold,
                    left=left_child, right=right_child)

    def _best_split(self, X, y, n_features):
        best_mse = float('inf')
        best_feature, best_threshold = None, None

        if self.max_features is not None and self.max_features < n_features:
            feature_indices = np.random.choice(n_features, size=self.max_features, replace=False)
        else:
            feature_indices = range(n_features)

        for feature_idx in feature_indices:
            thresholds = np.unique(X[:, feature_idx])
            for thr in thresholds:
                left_idxs, right_idxs = self._split(X[:, feature_idx], thr)
                if len(left_idxs) == 0 or len(right_idxs) == 0:
                    continue
                mse = self._calculate_mse(y[left_idxs], y[right_idxs])
                if mse < best_mse:
                    best_mse = mse
                    best_feature = feature_idx
                    best_threshold = thr
        return best_feature, best_threshold

    def _split(self, X_column, threshold):
        left_idxs = np.where(X_column <= threshold)[0]
        right_idxs = np.where(X_column > threshold)[0]
        return left_idxs, right_idxs

    def _calculate_mse(self, y_left, y_right):
        n_left, n_right = len(y_left), len(y_right)
        if n_left == 0 or n_right == 0:
            return float('inf')
        mse_left = np.var(y_left) * n_left
        mse_right = np.var(y_right) * n_right
        return (mse_left + mse_right) / (n_left + n_right)

    def predict(self, X):
        X = np.array(X, dtype=np.float64)
        return np.array([self._predict_one(x) for x in X])

    def _predict_one(self, x):
        node = self.root
        while node.value is None:
            if x[node.feature] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value

In [41]:
class MyGradientBoostingClassifier:
    def __init__(
        self,
        n_estimators=100,
        learning_rate=0.1,
        max_depth=10,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features=None,
        random_state=None
    ):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.random_state = random_state
        self.trees_per_class = []
        self.classes_ = None
        self.n_classes_ = None

    def _softmax(self, F):
        F = np.clip(F, -250, 250)
        exp_F = np.exp(F - np.max(F, axis=1, keepdims=True))
        return exp_F / np.sum(exp_F, axis=1, keepdims=True)

    def fit(self, X, y):
        X = np.array(X, dtype=np.float64)
        y = np.array(y)
        if self.random_state is not None:
            np.random.seed(self.random_state)

        self.classes_ = np.unique(y)
        self.n_classes_ = len(self.classes_)

        n_samples, n_features = X.shape

        F = np.zeros((n_samples, self.n_classes_), dtype=np.float64)

        y_one_hot = np.zeros((n_samples, self.n_classes_), dtype=np.float64)
        for idx, cls in enumerate(self.classes_):
            y_one_hot[:, idx] = (y == cls).astype(np.float64)

        self.trees_per_class = [[] for _ in range(self.n_classes_)]

        for _ in range(self.n_estimators):
            probas = self._softmax(F)

            residuals = y_one_hot - probas

            for k in range(self.n_classes_):
                tree = MyDecisionTreeRegressor(
                    max_depth=self.max_depth,
                    min_samples_split=self.min_samples_split,
                    min_samples_leaf=self.min_samples_leaf,
                    max_features=self.max_features
                )
                tree.fit(X, residuals[:, k])
                self.trees_per_class[k].append(tree)

                F[:, k] += self.learning_rate * tree.predict(X)

    def predict_proba(self, X):
        X = np.array(X, dtype=np.float64)
        if X.ndim == 1:
            X = X.reshape(1, -1)

        n_samples = X.shape[0]
        F = np.zeros((n_samples, self.n_classes_), dtype=np.float64)

        for k in range(self.n_classes_):
            for tree in self.trees_per_class[k]:
                F[:, k] += self.learning_rate * tree.predict(X)

        return self._softmax(F)

    def predict(self, X):
        proba = self.predict_proba(X)
        class_indices = np.argmax(proba, axis=1)
        return self.classes_[class_indices]

In [42]:
grad_boost_class = MyGradientBoostingClassifier()

grad_boost_class.fit(class_x_train, class_y_train)
class_y_train_pred = grad_boost_class.predict(class_x_train)
class_y_test_pred = grad_boost_class.predict(class_x_test)

In [ ]:
print(classification_report(class_y_train, class_y_train_pred))
print(classification_report(class_y_test, class_y_test_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       760
           1       1.00      1.00      1.00      5593
           2       1.00      1.00      1.00      1647

    accuracy                           1.00      8000
   macro avg       1.00      1.00      1.00      8000
weighted avg       1.00      1.00      1.00      8000

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       201
           1       1.00      1.00      1.00      1396
           2       1.00      1.00      1.00       403

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



Градиентный бустинг для задачи классификации, написанный мной, прекрасно справилсь со своей задачей и хорошо разделил все объекты на классы. Как и модель из библиотеки sklearn.

Единственный минус моего решения - оно работает медленнее модели из sklearn. На то, чтобы обучиться у него ушло около 10 минут.

### Регрессия

In [45]:
import numpy as np

class MyGradientBoostingRegressor:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=5,
                 min_samples_split=2, min_samples_leaf=1, max_features=None):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.trees = []
        self.initial_prediction = None

    def fit(self, X, y):
        X, y = np.array(X, dtype=np.float64), np.array(y, dtype=np.float64)
        n_samples = X.shape[0]

        self.initial_prediction = np.mean(y)
        y_pred = np.full(n_samples, self.initial_prediction, dtype=np.float64)

        for _ in range(self.n_estimators):
            residuals = y - y_pred

            tree = MyDecisionTreeRegressor(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                min_samples_leaf=self.min_samples_leaf,
                max_features=self.max_features
            )
            tree.fit(X, residuals)
            self.trees.append(tree)

            update = tree.predict(X)
            y_pred += self.learning_rate * update

    def predict(self, X):
        X = np.array(X, dtype=np.float64)
        y_pred = np.full(X.shape[0], self.initial_prediction, dtype=np.float64)
        for tree in self.trees:
            y_pred += self.learning_rate * tree.predict(X)
        return y_pred

In [46]:
grad_boost_regressor = MyGradientBoostingRegressor(n_estimators=70)

grad_boost_regressor.fit(reg_x_train, reg_y_train)
reg_y_train_pred = grad_boost_regressor.predict(reg_x_train)
reg_y_test_pred = grad_boost_regressor.predict(reg_x_test)

In [47]:
mse_train = mean_squared_error(reg_y_train_pred, reg_y_train)
mse_test = mean_squared_error(reg_y_test_pred, reg_y_test)
print(f"Среднеквадратичная ошибка трейна: {mse_train}")
print(f"Среднеквадратичная ошибка теста: {mse_test}")

mae_train = mean_absolute_error(reg_y_train_pred, reg_y_train)
mae_test = mean_absolute_error(reg_y_test_pred, reg_y_test)
print(f"Средняя абсолютная ошибка трейна: {mae_train}")
print(f"Средняя абсолютная ошибка теста: {mae_test}")

Среднеквадратичная ошибка трейна: 0.20221173148891808
Среднеквадратичная ошибка теста: 0.2506726863495539
Средняя абсолютная ошибка трейна: 0.36666416680318914
Средняя абсолютная ошибка теста: 0.4083013433104336


В целом, моя реализация градиентного бустинга хорошо соотносится с реализацией модели в sklearn.

Правда, обучается и предсказывает медленнее.

## Выводы

### Классификация

KNN-модель смогла достаточно точно разделить данные, пусть и была небольшая ошибка.

Все остальные модели (логистическая регрессия, решающее дерево, случайный лес, градиентный бустинг) сомогли разделить данные безошибочно. Думаю, это связано с тем, что датасет был создан синтетически по какому-то определенному закону или с учетом некоторых закономерностей.

### Регрессия

Хадшей по предсказанию моделью (с учетом всех улучшений) была линейная регрессия (Она уступает несколько десятых).

Все остальные модели по MAE (абсолютной ошибке) предсказывают одинаково (даже KNN-модель).

По MSE одинаково лучшими были решающее дерево, случайный лес и градиентный бустинг.

### Общие выводы

KNN - самая простая в понимании и кодинге модель. При этом она тоже может неплохо справляться со своей задачей и давать адекватные результаты.

Линейная и логистическая регрессия - достаточно простые математические модели, то, что следует обучать и проверять в первую очередь. При этом для этих моделей важно хорошо почистить датасет: убратить линейнозависимые признаки, убрать выбросы, провести нормализацию данных, чтобы все признаки были одинаково важными.

Модель решающего дерева основа на построении бинарного дерева. Если не ограничить глубину дерева, то оно подстроиться под тренировочные данные и в каждом листе будет условно по одному значению таргета. Это приведет к переобучени.

Случайный лес обучает несколько решающих деревьев, каждое дерево на своей рандомно выбранной части признаков. Тем самым устраняется недостаток в виде переобучения. Увеличение деревьев в ансамбле не ведет к переобучению. Однако при обучении очень большого количества решающих деревьев могут вознукнуть проблемы с производительностью: на обучение будет потрачено много времени.

Градиентный бустинг так же объединяет в себе несколько решающих деревьев (хотя можно выбрать другой базовый алгоритм). Каждое следующее дерево пытается исправить ошибки предыдущих. На каждом шаге вес сохраняется. А предсказание строится как композиция предказаний отдельных деревьев с соответствующими им весами. При этом времени на обучение тратиться еще больше, чем в случайном лесе.

Таким образом, при решении задачи важно понимать, чем можно пожертвовать и что более важно: скорость или результат.